In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict # Muy rápido
import os
import json
import math

MIN_BUSINESSES_PER_CATEGORY = 15 # Para limpieza y reduccion de dimensionalidad
C = 12 # Para el promedio Bayesiano de confianza
ALPHA = 0.3 #Para el Social Boost 
OUTPUT_FILE = "a1_users_profile.json"

cwd = os.getcwd()
businesses_path = cwd+"/a11_cleaned_businesses.json"
reviews_path = cwd+"/a1_filtered_reviewsv3.json"
users_path = cwd+"/a1_cleaned_users.json"


businesses = pd.read_json(businesses_path, lines=True)
reviews = pd.read_json(reviews_path, lines=True)
users = pd.read_json(users_path, lines=True)


In [4]:
businesses["categories"] = businesses["categories"].str.split(", ")

category_counts = defaultdict(int)
for cats in businesses["categories"]:
    if isinstance(cats, list):
        for cat in cats:
            category_counts[cat] += 1


valid_categories = {cat for cat, count in category_counts.items() 
                    if count >= MIN_BUSINESSES_PER_CATEGORY}

print(f"Categorias finales {len(valid_categories)}")

Updated categories 983


In [6]:
business_cat_map = {}
for _, row in businesses.iterrows():
    if row["categories"]:
        filtered_cats = [cat for cat in row["categories"] if cat in valid_categories]
        if filtered_cats:  
            business_cat_map[row["business_id"]] = filtered_cats

In [8]:
# IDF para categorias filtradas
category_users = defaultdict(set)
for chunk in pd.read_json(reviews_path, lines=True, chunksize=100000):
    for _, row in chunk.iterrows():
        user_id = row["user_id"]
        business_id = row["business_id"]
        categories = business_cat_map.get(business_id, [])
        for cat in categories:
            if cat in valid_categories:
                category_users[cat].add(user_id)

In [10]:
total_users = pd.read_json(users_path, lines=True)["user_id"].nunique()
idf_dict = {cat: np.log(total_users / (len(users) + 1)) 
            for cat, users in category_users.items()}

In [12]:
global_avg = pd.read_json(reviews_path, lines=True)["stars"].mean()

In [13]:
# Perfiles Base antes de SocialBoost
def calculate_basic_profile(user_reviews):
    user_avg = user_reviews["stars"].mean()
    total_reviews = len(user_reviews)
    category_stats = defaultdict(lambda: {"sum_adj": 0, "count": 0})

    for _, review in user_reviews.iterrows():
        business_id = review["business_id"]
        rating = review["stars"]
        categories = business_cat_map.get(business_id, [])
        adj_rating = (rating - user_avg) + global_avg
        for cat in categories:
            if cat in valid_categories:
                category_stats[cat]["sum_adj"] += adj_rating
                category_stats[cat]["count"] += 1

    weights = {}
    for cat, stats in category_stats.items():
        count = stats["count"]
        adj_rating = stats["sum_adj"] / count if count > 0 else 0
        confidence = count / (count + C)
        tf = count / total_reviews if total_reviews > 0 else 0
        idf = idf_dict.get(cat, 1.0)
        weights[cat] = adj_rating * confidence * tf * idf
    
    return weights

basic_profiles = {}
for chunk in pd.read_json(reviews_path, lines=True, chunksize=100000):
    for user_id, group in chunk.groupby("user_id"):
        basic_profiles[user_id] = calculate_basic_profile(group)

In [14]:
# Perfiles amigos y suma del SocialBoost - GPT
friends_map = users.set_index("user_id")["friends"].to_dict()

enhanced_profiles = {}
for user_id, profile in basic_profiles.items():
    friends = friends_map.get(user_id, [])
    friend_profiles = [basic_profiles.get(f, {}) for f in friends]
    
    social_boost = defaultdict(float)
    for friend_profile in friend_profiles:
        for cat, weight in friend_profile.items():
            social_boost[cat] += weight
    
    for cat in social_boost:
        social_boost[cat] = ALPHA * social_boost[cat] / len(friends) if friends else 0
    
    enhanced_profile = {
        cat: profile.get(cat, 0) + social_boost.get(cat, 0)
        for cat in set(profile.keys()) | set(social_boost.keys())
    }
    enhanced_profiles[user_id] = enhanced_profile

output_data = [{"user_id": uid, "profile": profile} 
              for uid, profile in enhanced_profiles.items()]

with open(OUTPUT_FILE, "w") as f:
    json.dump(output_data, f, indent=2)

print(f"Saved {len(enhanced_profiles)} user profiles to {OUTPUT_FILE}")

Saved 228195 user profiles to a1_users_profile.json


In [15]:
with open("a1_users_profile.json") as f:
    user_profiles = json.load(f)

# Convert to DataFrame if needed
profiles_df = pd.DataFrame([
    {"user_id": x["user_id"], **x["profile"]} 
    for x in user_profiles
]).fillna(0)

In [20]:
#Los pesos se normalizan usando L2
def l2_normalize_profiles(input_file, output_file):
    with open(input_file, 'r') as f:
        users = json.load(f)
    
    for user in users:
        profile = user['profile']
        
        # Norma L2
        squared_sum = sum(value ** 2 for value in profile.values())
        l2_norm = math.sqrt(squared_sum)
        
        if l2_norm > 0:
            for category in profile:
                profile[category] /= l2_norm
        else:
            print(f"Warning: Zero norm for user {user['user_id']} - profile not modified")
    
    # Se guardan solo los pesos de categorias diferentes de 0
    with open(output_file, 'w') as f:
        json.dump(users, f, indent=2)
    
    print(f"Normalization complete. Saved to {output_file}")

# Usage
input_filename = "a1_users_profile.json"
output_filename = "a1_users_profile_l2_normalized.json"
l2_normalize_profiles(input_filename, output_filename)

Normalization complete. Saved to a1_users_profile_l2_normalized.json
